In [136]:
import os
print(os.listdir("../input"))

['to-be-what-testcases', 'corpus.txt']


In [137]:
import numpy as np
import re
from nltk.util import ngrams
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import fileinput

In [138]:
raw_corpus = open('../input/corpus.txt', 'r', encoding='utf-8-sig').read()

In [139]:
title_match_regex = '\n{3,}\s+\w+\n{3,}.*' # used to remove headers, toc, etc.
corpus = re.search(title_match_regex, raw_corpus, flags=re.M+re.S).group()

In [140]:
#corpus

In [141]:
corpus = corpus.replace('\n', ' ') 
corpus = re.sub(r' {2,}', ' ', corpus) # replace multiple blanks by one
corpus = corpus.replace('----', '') # remove consecutive hyphens that we'll as a tag for the be verb

In [142]:
corpus = sent_tokenize(corpus)

In [143]:
corpus =  corpus[37:]
corpus [:5]

['The fresh wind reddened his cheeks and the roaring of the rapids filled his ears.',
 'Eagerly his eyes followed the movements of the canoes daringly poised in the stream just below the tossing, foaming, white water.',
 'It was the first day of the spring fishing, and more exciting sport than this Indian white-fishing Hugh Beaupré had never seen.',
 'Three canoes were engaged in the fascinating game, two Indians in each.',
 'One knelt in the stern with his paddle.']

In [144]:
tobe_verbs = ['am','are','were','was','is','been','being','be']
substitute = '----'


In [145]:
def remove_targets_verbs(tokens):
    return [substitute if token in tobe_verbs else token for token in tokens]

In [146]:

def remove_punctualtions(word_list):
    # define punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # remove punctuation from the string
    no_punct = ""
    for char in word_list:
       if char not in punctuations:
           no_punct = no_punct + char
    return(no_punct)        

In [147]:
print(remove_punctualtions("Hello!!!, he said ---and went."))

Hello he said and went


In [148]:
def find_targets_verbs(tokens):
    return [token for token in tokens if token in tobe_verbs]

In [149]:
all_corpus_tokens = []
n_grams = []
window_size = 5
all_targeted_verbs = [] 
for sentence in corpus:
    #print(sentence, end="\n\n" )
    sent = sentence.lower()
    sent_wothout_punct = remove_punctualtions(sent)
    tokens_with_tobe_verbs = word_tokenize(sent_wothout_punct)
    targeted_verbs = find_targets_verbs(tokens_with_tobe_verbs)
    if len(targeted_verbs)>0:
        print(targeted_verbs)
        for verb in targeted_verbs:
            all_targeted_verbs.append(verb)
        
    tokens_without_tobe_verbs = remove_targets_verbs(tokens_with_tobe_verbs)
    if(set(tokens_with_tobe_verbs) & set(tobe_verbs)):
        print(tokens_without_tobe_verbs)
        for token in tokens_without_tobe_verbs:
            all_corpus_tokens.append(token)

['was']
['it', '----', 'the', 'first', 'day', 'of', 'the', 'spring', 'fishing', 'and', 'more', 'exciting', 'sport', 'than', 'this', 'indian', 'whitefishing', 'hugh', 'beaupré', 'had', 'never', 'seen']
['were']
['three', 'canoes', '----', 'engaged', 'in', 'the', 'fascinating', 'game', 'two', 'indians', 'in', 'each']
['was']
['the', 'standing', 'indian', 'in', 'the', 'nearest', 'canoe', '----', 'a', 'fine', 'figure', 'of', 'a', 'young', 'man', 'in', 'closefitting', 'buckskin', 'leggings', 'his', 'slender', 'muscular', 'bronze', 'body', 'stripped', 'to', 'the', 'waist']
['been']
['the', 'lad', 'on', 'the', 'bank', 'had', '----', 'holding', 'his', 'breath']
['was', 'be']
['“', 'holá', 'hugh', 'beaupré', '”', 'he', 'cried', 'when', 'he', '----', 'close', 'enough', 'to', '----', 'heard', 'above', 'the', 'tumult', 'of', 'the', 'rapids']
['be']
['he', 'has', 'brought', 'news', 'of', 'my', 'father', '”', '“', 'that', 'm', '’', 'sieu', 'did', 'not', 'tell', 'me', 'but', 'yes', 'i', 'think', 'it'

['were']
['on', 'the', 'whole', 'hugh', 'and', 'blaise', '----', 'lucky', 'during', 'the', 'first', 'part', 'of', 'their', 'trip', 'and', 'they', 'reached', 'the', 'pigeon', 'river', 'in', 'good', 'time']
['was', 'were']
['hugh', '----', 'gaining', 'experience', 'and', 'his', 'paddling', 'muscles', '----', 'hardening']
['be']
['he', 'would', 'soon', '----', 'able', 'he', 'felt', 'to', 'hold', 'his', 'own', 'easily', 'at', 'any', 'pace', 'his', 'halfbrother', 'set']
['be']
['so', 'far', 'blaise', 'had', 'proved', 'a', 'good', 'travelling', 'companion', 'somewhat', 'silent', 'and', 'grave', 'to', '----', 'sure', 'but', 'dependable', 'patient', 'and', 'for', 'the', 'most', 'part', 'even', 'tempered']
['was']
['blaise', 'the', 'older', 'boy', 'decided', '----', 'in', 'spite', 'of', 'his', 'quebec', 'training', 'and', 'many', 'civilized', 'ways', 'more', 'indian', 'than', 'french']
['was']
['satisfactory', 'comrade', 'though', 'the', 'halfbreed', 'boy', 'seemed', 'hugh', '----', 'in', 'no',

['the', 'sun', 'had', 'not', 'yet', 'risen', 'but', 'the', 'morning', '----', 'clear', 'of', 'fog', 'or', 'haze']
['was']
['hugh', '’', 's', 'attention', '----', 'attracted', 'to', 'the', 'man', 'in', 'the', 'stern']
['was']
['indeed', 'that', 'man', '----', 'too', 'notable', 'and', 'unusual', 'a', 'figure', 'to', 'escape', 'attention']
['were']
['a', 'long', 'eagle', 'feather', 'upright', 'from', 'the', 'band', 'about', 'his', 'head', 'made', 'him', 'appear', 'still', 'taller', 'while', 'his', 'huge', 'shoulders', 'and', 'bigmuscled', 'arms', '----', 'conspicuous', 'as', 'he', 'wielded', 'his', 'paddle', 'on', 'the', 'left', 'side', 'of', 'the', 'canoe']
['was', 'was']
['by', 'the', 'time', 'camp', '----', 'reached', 'the', 'elder', 'brother', '----', 'almost', 'bursting', 'with', 'curiosity']
['was', 'been']
['who', '----', 'the', 'huge', 'indian', 'and', 'why', 'had', 'blaise', '----', 'so', 'startled', 'even', 'frightened', 'at', 'the', 'sight', 'of', 'him']
['is']
['“', 'who', '--

['“', 'it', '----', 'the', 'mirage']
['is', 'is']
['the', 'captain', 'of', 'the', 'athabasca', 'told', 'me', 'that', 'the', 'mirage', '----', 'caused', 'by', 'the', 'light', 'shining', 'through', 'mist', 'or', 'layers', 'of', 'cloud', 'or', 'air', 'that', 'reflect', 'in', 'some', 'way', 'we', 'do', 'not', 'understand', 'making', 'images', 'of', 'land', 'appear', 'where', 'there', '----', 'no', 'land', 'or', 'changing', 'the', 'appearance', 'of', 'the', 'real', 'land']
['are']
['sometimes', 'he', 'said', 'images', 'of', 'islands', '----', 'seen', 'upside', 'down', 'in', 'the', 'sky', 'above', 'the', 'real', 'waterline']
['is', 'is']
['it', '----', 'all', 'very', 'strange', 'and', 'no', 'one', 'quite', 'understands', 'why', 'it', 'comes', 'or', 'how', 'but', 'there', '----', 'no', 'enchantment', 'about', 'it', 'blaise', '”', 'the', 'younger', 'boy', 'nodded', 'his', 'eyes', 'still', 'on', 'the', 'changing', 'hazy', 'shapes', 'ahead']
['was', 'be']
['at', 'any', 'rate', 'blaise', '----', 

['off', 'the', 'shore', 'and', 'in', 'the', 'channels', 'of', 'minong', 'he', 'has', 'set', 'sharp', 'rocks', 'to', 'destroy', 'the', 'canoes', 'that', 'approach', 'the', 'island', 'and', 'he', 'has', 'many', 'spirits', 'to', 'help', 'him', 'guard', 'the', 'treasure', '”', '“', 'that', '----', 'only', 'a', 'tale', 'of', 'course', '”', 'said', 'hugh', 'somewhat', 'disdainfully']
['was']
['we', 'found', 'nothing', 'to', 'fear', '”', '“', 'you', 'sought', 'no', 'copper', '”', '----', 'the', 'retort']
['is']
['“', 'it', '----', 'said', 'that', 'sometimes', 'kepoochikan', 'and', 'nanibozho', 'fight', 'together', 'on', 'the', 'rocks', 'and', 'hurl', 'great', 'boulders', 'about']
['are']
['strange', 'tales', 'there', '----', 'too', 'of', 'the', 'thick', 'forest', 'of', 'the', 'little', 'lakes', 'and', 'bays']
['is']
['there', '----', 'one', 'place', 'called', 'the', 'bay', 'of', 'manitos', 'where', 'so', 'i', 'have', 'heard', 'dwell', 'giant', 'windigos', 'and', 'great', 'serpents', 'and', 'h

...

['beyond', 'the', 'obvious', 'facts', 'that', 'he', 'has', 'at', 'some', 'time', 'done', 'manual', 'labour', 'that', 'he', 'takes', 'snuff', 'that', 'he', '----', 'a', 'freemason', 'that', 'he', 'has', '----', 'in', 'china', 'and', 'that', 'he', 'has', 'done', 'a', 'considerable', 'amount', 'of', 'writing', 'lately', 'i', 'can', 'deduce', 'nothing', 'else']
['is']
['your', 'right', 'hand', '----', 'quite', 'a', 'size', 'larger', 'than', 'your', 'left']
['are']
['you', 'have', 'worked', 'with', 'it', 'and', 'the', 'muscles', '----', 'more', 'developed']
['be']
['what', 'else', 'can', '----', 'indicated', 'by', 'that', 'right', 'cuff', 'so', 'very', 'shiny', 'for', 'five', 'inches', 'and', 'the', 'left', 'one', 'with', 'the', 'smooth', 'patch', 'near', 'the', 'elbow', 'where', 'you', 'rest', 'it', 'upon', 'the', 'desk']
['been']
['the', 'fish', 'that', 'you', 'have', 'tattooed', 'immediately', 'above', 'your', 'right', 'wrist', 'could', 'only', 'have', '----', 'done', 'in', 'china']
['is

['are']
['they', '----', 'typewritten', 'i', 'remarked']
['is']
['not', 'only', 'that', 'but', 'the', 'signature', '----', 'typewritten']
['is', 'is']
['there', '----', 'a', 'date', 'you', 'see', 'but', 'no', 'superscription', 'except', 'leadenhall', 'street', 'which', '----', 'rather', 'vague']
['is']
['the', 'point', 'about', 'the', 'signature', '----', 'very', 'suggestivein', 'fact', 'we', 'may', 'call', 'it', 'conclusive']
['is']
['my', 'dear', 'fellow', '----', 'it', 'possible', 'you', 'do', 'not', 'see', 'how', 'strongly', 'it', 'bears', 'upon', 'the', 'case']
['were', 'be', 'were']
['i', 'can', 'not', 'say', 'that', 'i', 'do', 'unless', 'it', '----', 'that', 'he', 'wished', 'to', '----', 'able', 'to', 'deny', 'his', 'signature', 'if', 'an', 'action', 'for', 'breach', 'of', 'promise', '----', 'instituted']
['was']
['no', 'that', '----', 'not', 'the', 'point']
['is', 'is']
['one', '----', 'to', 'a', 'firm', 'in', 'the', 'city', 'the', 'other', '----', 'to', 'the', 'young', 'ladys'

['were']
['we', '----', 'to', 'meet', 'at', 'the', 'pool', 'midway', 'between', 'our', 'houses', 'to', 'talk', 'it', 'over']
['be']
['when', 'i', 'went', 'down', 'there', 'i', 'found', 'him', 'talking', 'with', 'his', 'son', 'so', 'i', 'smoked', 'a', 'cigar', 'and', 'waited', 'behind', 'a', 'tree', 'until', 'he', 'should', '----', 'alone']
['was']
['but', 'as', 'i', 'listened', 'to', 'his', 'talk', 'all', 'that', '----', 'black', 'and', 'bitter', 'in', 'me', 'seemed', 'to', 'come', 'uppermost']
['was', 'were']
['he', '----', 'urging', 'his', 'son', 'to', 'marry', 'my', 'daughter', 'with', 'as', 'little', 'regard', 'for', 'what', 'she', 'might', 'think', 'as', 'if', 'she', '----', 'a', 'slut', 'from', 'off', 'the', 'streets']
['be']
['it', 'drove', 'me', 'mad', 'to', 'think', 'that', 'i', 'and', 'all', 'that', 'i', 'held', 'most', 'dear', 'should', '----', 'in', 'the', 'power', 'of', 'such', 'a', 'man', 'as', 'this']
['was']
['i', '----', 'already', 'a', 'dying', 'and', 'a', 'desperate'

['you', 'must', '----', 'weary', 'for', 'you', 'have', 'had', 'a', 'long', 'day']
['is']
['this', '----', 'my', 'friend', 'dr', 'watson']
['been']
['he', 'has', '----', 'of', 'most', 'vital', 'use', 'to', 'me', 'in', 'several', 'of', 'my', 'cases', 'and', 'a', 'lucky', 'chance', 'has', 'made', 'it', 'possible', 'for', 'me', 'to', 'bring', 'him', 'out', 'and', 'associate', 'him', 'with', 'this', 'investigation']
['am']
['i', '----', 'delighted', 'to', 'see', 'you', 'said', 'she', 'pressing', 'my', 'hand', 'warmly']
['am', 'be']
['you', 'will', 'i', '----', 'sure', 'forgive', 'anything', 'that', 'may', '----', 'wanting', 'in', 'our', 'arrangements', 'when', 'you', 'consider', 'the', 'blow', 'which', 'has', 'come', 'so', 'suddenly', 'upon', 'us']
['am', 'were', 'is']
['my', 'dear', 'madam', 'said', 'i', 'i', '----', 'an', 'old', 'campaigner', 'and', 'if', 'i', '----', 'not', 'i', 'can', 'very', 'well', 'see', 'that', 'no', 'apology', '----', 'needed']
['be', 'be']
['if', 'i', 'can', '----

['was']
['sometimes', 'i', 'have', 'thought', 'that', 'it', '----', 'merely', 'the', 'wild', 'talk', 'of', 'delirium', 'sometimes', 'that', 'it', 'may', 'have', 'referred', 'to', 'some', 'band', 'of', 'people', 'perhaps', 'to', 'these', 'very', 'gipsies', 'in', 'the', 'plantation']
['is', 'being']
['holmes', 'shook', 'his', 'head', 'like', 'a', 'man', 'who', '----', 'far', 'from', '----', 'satisfied']
['are']
['these', '----', 'very', 'deep', 'waters', 'said', 'he', 'pray', 'go', 'on', 'with', 'your', 'narrative']
['been']
['two', 'years', 'have', 'passed', 'since', 'then', 'and', 'my', 'life', 'has', '----', 'until', 'lately', 'lonelier', 'than', 'ever']
['is']
['his', 'name', '----', 'armitagepercy', 'armitagethe', 'second', 'son', 'of', 'mr', 'armitage', 'of', 'crane', 'water', 'near', 'reading']
['are', 'be']
['my', 'stepfather', 'has', 'offered', 'no', 'opposition', 'to', 'the', 'match', 'and', 'we', '----', 'to', '----', 'married', 'in', 'the', 'course', 'of', 'the', 'spring']
['

In [150]:
def create_windows(tokens, window_size = 5):
    """ Creating ngram windows surrouding be forms i.e. substitute . """
    contexts = []
    for i, word in enumerate(tokens):
        if word == substitute:
            #print(substitute)
            window = tokens[i-window_size:i] + tokens[i+1:i+window_size+1]
            window = ' '.join(window)
            contexts.append(window)    
    return contexts


In [151]:
print(len(all_corpus_tokens))

327360


In [152]:
contexts = create_windows(all_corpus_tokens)


In [153]:
print(len(contexts))
print(len(all_targeted_verbs))

18746
18746


In [154]:
all_targeted_verbs[:5]

['was', 'were', 'was', 'been', 'was']

In [174]:
counts = np.unique(all_targeted_verbs, return_counts=True)
print(counts)

(array(['am', 'are', 'be', 'been', 'being', 'is', 'was', 'were'],
      dtype='<U5'), array([ 261, 1070, 2493, 1407,  334, 2899, 8028, 2254]))


In [155]:
#Label Encoder
label_encoder = LabelEncoder()
model = label_encoder.fit_transform(all_targeted_verbs)

In [158]:
#Building A Model
X_train, X_test, Y_train, Y_test = train_test_split(contexts, model, test_size=0.3)

In [161]:
vectorizer = CountVectorizer()
classifier = LogisticRegression()
pipe = make_pipeline(vectorizer, classifier)
pipe.fit(X_train, Y_train)


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [162]:
y_pred = pipe.predict(X_test)

In [171]:

sample = open('../input/to-be-what-testcases/input/input02.txt', 'r', encoding='utf-8-sig').read().splitlines()
N = int(sample[0])
sample = sample[1]


In [172]:
sample = wordpunct_tokenize(sample)
contexts_sample = create_windows(sample)
sample_pred = label_encoder.inverse_transform(pipe.predict(contexts_sample))
print('\n'.join(sample_pred))

was
was
was
was
was
were


In [173]:
Output = open('../input/to-be-what-testcases/output/output02.txt', 'r', encoding='utf-8-sig').read().splitlines()

Output

['was', 'were', 'were', 'is', 'been', 'am']